In [206]:
import pandas as pd
import nltk
import pymorphy2
from collections import Counter
from tqdm.notebook import tqdm
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import re
from nltk.stem import WordNetLemmatizer
import numpy as np
from sklearn.decomposition import LatentDirichletAllocation
from operator import itemgetter
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [202]:
clusters_num = 20
text = pd.read_excel('/Users/thebest/Desktop/python/Учеба/курсовая работа /sciencedir_title.xlsx').drop(columns = ['Unnamed: 0'])

text1 = text['Заголовок']


tokens_list_1 = []
for token in text1:
    nltk.word_tokenize(token)
    tokens_list_1.append(token.lower())
x  = str(tokens_list_1)
tokens_list_all = nltk.word_tokenize(x)

tokens_list_2 = []
for token in tokens_list_all:
    if re.search('\w', token) != None:
        tokens_list_2.append(token)

tag_list = ['JJ', 'JJR', 'JJS', 'NN', 'NNS', 'NNP', 'NNPS', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']

tokens_list_3 = []
tokens_title_sent = []
for x in nltk.pos_tag(tokens_list_2):
    if x[1] in tag_list:
        if x[1] in ['JJ', 'JJR', 'JJS']:
            tokens_list_3.append(nltk.stem.WordNetLemmatizer().lemmatize(x[0], pos = 'a'))
        elif x[1] in ['NN', 'NNS', 'NNP', 'NNPS']:
            tokens_list_3.append(nltk.stem.WordNetLemmatizer().lemmatize(x[0], pos = 'n'))
        else:
            tokens_list_3.append(nltk.stem.WordNetLemmatizer().lemmatize(x[0], pos = 'v'))
    
# pd.DataFrame(dict(Counter(tokens_list_3)).items()).to_excel('/Users/thebest/Desktop/python/Учеба/курсовая работа /результат/new/bag_of_words_title.xlsx')

In [203]:
texts_lem_lists = []
for sentece_lem_list in (tokens_list_3):
    texts_lem_lists.append(''.join(sentece_lem_list))    
    
tfidfconverter = TfidfVectorizer(max_features = 1500)
X = tfidfconverter.fit_transform(texts_lem_lists).toarray()

lda = LatentDirichletAllocation(n_components = clusters_num,
                                learning_method = "batch",
                                max_iter = 25,
                                random_state = 0)
document_topics = lda.fit_transform(X)

sorting = np.argsort(lda.components_, axis = 1)[:, ::-1]
feature_names = np.array(list(dict(sorted(tfidfconverter.vocabulary_.items(), key = itemgetter(1))).keys()))   
clusters_tokens = []
for num in (range (0, clusters_num)):
    tokens_list = list(feature_names[list(sorting[num])])
    clusters_tokens.append(tokens_list)   
df_clust_title = pd.DataFrame(clusters_tokens).T

In [204]:
df_clust_title

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,do,19,currency,discovery,financial,trading,stock,portfolio,cryptocurrency,analysis,bubble,future,market,bitcoin,volatility,energy,efficiency,price,use,review
1,digital,covid,blockchain,approach,safe,high,be,effect,pandemic,impact,predictability,technology,investor,evidence,asset,defi,liquidity,cryptocurrencies,risk,perspective
2,case,regulatory,return,model,money,security,arbitrage,information,fintech,opportunity,monetary,regulation,learning,exchange,dynamic,tweet,sentiment,memory,multiple,spillover
3,scale,multifractal,time,research,power,renewable,new,hedge,ethereum,break,learn,haven,finance,transformation,gold,asymmetric,frequency,network,real,global
4,designing,event,momentum,attention,crisis,order,study,volume,economic,artificial,brazilian,commodity,long,sector,pricing,connectedness,relationship,corporate,industrial,performance
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
657,equilibrium,great,inflow,dynamic,response,green,scale,governance,safe,herding,tweet,green,novel,offering,herding,discovery,revolution,short,deep,robotics
658,central,green,information,do,respond,scale,multiple,great,identify,high,asymmetric,grid,nonlinear,nrgcoin,high,digital,frame,evolution,negative,equilibrium
659,asymmetric,response,section,revolution,research,multiple,day,green,run,cryptocurrency,designing,have,noise,novel,hold,review,framework,shock,scale,run
660,industrial,have,seasonality,diversification,extract,search,search,grid,impact,cryptocurrencies,energy,revolution,nfts,nonlinear,horizon,revolution,review,expected,study,safe


In [211]:
# 2 - Сезонность торговли криптовалютами
# 4  - безопасность финансового сектора
# 9 - анализ возможностей торговли криптовалютами
# 12 - инвестиционный рынок
# 14 - Динамика волатильности цен на торговые позиции
# 17 - эволюция цен на криптовалюты
# 11 - технологии регулирования рынка в будущем
# 8 - влияние пандемии на рынок криптовалют

texts = list(text['Заголовок'])[:-200]
texts_test = list(text['Заголовок'])[-200:]


clusters_norm_nums = [2,4,8,9,11,12,14,17]

clusters_solution = []
for marks_list in tqdm(document_topics):
    clusters_solution.append(marks_list[clusters_norm_nums])
y = []
for marks_list in tqdm(clusters_solution):
    m = list(marks_list)
    y.append(m.index(max(m)))
    
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

classifier = RandomForestClassifier(n_estimators = 100, random_state = 0 )
classifier.fit(X_train, y_train)

y_pred = classifier.predict(X_test)
print(accuracy_score(y_test, y_pred))

scores = classifier.predict_proba(tfidfconverter.transform(texts_test).toarray())
df_lists = []
index = 0
for t in texts_test:
    df_lists.append(list(np.append(t,scores[index])))
    index += 1
pd.DataFrame(df_lists)
# pd.DataFrame(df_lists).to_excel('/Users/thebest/Desktop/python/Учеба/курсовая работа /результат/new/Осмысление_кластеры_new.xlsx')

  0%|          | 0/1648 [00:00<?, ?it/s]

  0%|          | 0/1648 [00:00<?, ?it/s]

0.7818181818181819


,0,1,2,3,4,5,6,7,8
0,An analysis model for detecting misbehaviors i...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Time-frequency co-movement of cryptocurrency r...,0.72,0.28,0.0,0.0,0.0,0.0,0.0,0.0
2,Is it worth to hold bitcoin?,0.0,0.08,0.0,0.15,0.0,0.0,0.77,0.0
3,The influence of stablecoin issuances on crypt...,0.64,0.36,0.0,0.0,0.0,0.0,0.0,0.0
4,Examining the Maturity of Bitcoin Price throug...,0.21,0.27,0.0,0.0,0.52,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
195,Index,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
196,Towards cross-commodity energy-sharing communi...,0.64,0.36,0.0,0.0,0.0,0.0,0.0,0.0
197,Index,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
198,Dependent relationships between Chinese commod...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
